In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
%matplotlib widget

In [100]:
import numpy as np
import matplotlib.pyplot as plt
import tqdm

import pycomlink as pycml

# Load example data and look at one CML

In [4]:
cml_list = pycml.io.examples.get_75_cmls()

100%|██████████| 75/75 [00:01<00:00, 62.67it/s]


In [5]:
cml = cml_list[0]
cml

<xarray.Dataset>
Dimensions:           (channel_id: 2, time: 2750)
Coordinates:
  * time              (time) datetime64[ns] 2017-06-28T00:00:10.241754112 ......
  * channel_id        (channel_id) object 'channel_1' 'channel_2'
    cml_id            |S17 b'MY1394_2_MY2336_4'
    site_a_latitude   float64 50.26
    site_b_latitude   float64 50.38
    site_a_longitude  float64 50.91
    site_b_longitude  float64 50.38
    frequency         (channel_id) float64 18.2 19.2
    polarization      |S1 b'V'
    length            float64 13.48
Data variables:
    tsl               (channel_id, time) float64 14.0 14.0 14.0 ... 14.0 13.0
    rsl               (channel_id, time) float64 -47.0 -47.0 ... -47.0 -47.0

In [6]:
cml.rsl.plot.line(x='time', figsize=(10,3));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Set default values to NaN and calculate TRSL
TRSL is the transmitted signal level minus the received signal level and represents the total path loss along the CML

In [7]:
for cml in cml_list:
    cml['tsl'] = cml.tsl.where(cml.tsl != 255.0)
    cml['rsl'] = cml.rsl.where(cml.rsl != -99.9)
    cml['trsl'] = cml.tsl - cml.rsl

In [8]:
cml = cml_list[0]
cml.trsl.plot.line(x='time', figsize=(10,3));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Do simple wet-dry classification using a rolling standard deviation 

In [9]:
threshold = 0.8

roll_std_dev = cml.trsl.rolling(time=60, center=True).std()
cml['wet'] = cml.trsl.rolling(time=60, center=True).std() > threshold

In [58]:
fig, axs = plt.subplots(2, 1, figsize=(10,3), sharex=True)

roll_std_dev.plot.line(x='time', ax=axs[0])
axs[0].axhline(threshold, color='k', linestyle='--')

cml.trsl.plot.line(x='time', ax=axs[1]);

# Get start and end of dry event
wet_start = np.roll(cml.wet, -1) & ~cml.wet
wet_end = np.roll(cml.wet, 1) & ~cml.wet

# Plot shaded area for each wet event
for wet_start_i, wet_end_i in zip(
    wet_start.isel(channel_id=0).values.nonzero()[0],
    wet_end.isel(channel_id=0).values.nonzero()[0],
):
    axs[1].axvspan(cml.time.values[wet_start_i], cml.time.values[wet_end_i], color='b', alpha=0.1)

axs[1].set_title('');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Determine baseline

In [60]:
cml['baseline'] = pycml.processing.baseline.baseline_constant(trsl=cml.trsl, wet=cml.wet, n_average_last_dry=5)

fig, ax = plt.subplots(figsize=(10,3))

cml.trsl.plot.line(x='time', alpha=0.5)
plt.gca().set_prop_cycle(None)
cml.baseline.plot.line(x='time');
plt.gca().set_prop_cycle(None)
plt.ylabel('TRSL');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Perform wet antenna estimation and compare to uncorrected baseline 

In [61]:
cml['waa'] = pycml.processing.wet_antenna.waa_schleiss_2013(
    rsl=cml.trsl, 
    baseline=cml.baseline, 
    wet=cml.wet, 
    waa_max=2.2, 
    delta_t=1, 
    tau=15,
)

In [90]:
fig, axs = plt.subplots(2, 1, figsize=(10,5), sharex=True)

plt.sca(axs[0])
cml.trsl.plot.line(x='time', alpha=0.5)
plt.gca().set_prop_cycle(None)
cml.baseline.plot.line(x='time', linestyle=':');
plt.gca().set_prop_cycle(None)
(cml.baseline + cml.waa).plot.line(x='time');
plt.ylabel('TRSL (dB)')

plt.sca(axs[1])
cml['A'] = cml.trsl - cml.baseline - cml.waa
cml['A'] = cml.A.where(cml.A > 0, 0)
cml['A_no_waa_correct'] = cml.trsl - cml.baseline
cml['A_no_waa_correct'] = cml.A_no_waa_correct.where(cml.A_no_waa_correct > 0, 0)
cml.A_no_waa_correct.plot.line(x='time', linestyle=':');
plt.gca().set_prop_cycle(None)
cml.A.plot.line(x='time');
plt.ylabel('path attenuation\nfrom rain (dB)');
axs[1].set_title('');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Calculate rain rate

In [88]:
cml['R'] = pycml.processing.k_R_relation.calc_R_from_A(A=cml.A, L_km=cml.length, f_GHz=cml.frequency)

In [95]:
fig, ax = plt.subplots(figsize=(10,3))
cml.R.plot.line(x='time');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Now do the whole processing for all CMLs 

In [103]:
for cml in tqdm.notebook.tqdm(cml_list):
    cml['wet'] = cml.trsl.rolling(time=60, center=True).std() > 0.8
    cml['baseline'] = pycml.processing.baseline.baseline_constant(
        trsl=cml.trsl, 
        wet=cml.wet, 
        n_average_last_dry=5,
    )
    cml['waa'] = pycml.processing.wet_antenna.waa_schleiss_2013(
        rsl=cml.trsl, 
        baseline=cml.baseline, 
        wet=cml.wet, 
        waa_max=2.2, 
        delta_t=1, 
        tau=15,
    )
    cml['A'] = cml.trsl - cml.baseline - cml.waa
    cml['R'] = pycml.processing.k_R_relation.calc_R_from_A(A=cml.A, L_km=cml.length, f_GHz=cml.frequency)


  0%|          | 0/75 [00:00<?, ?it/s]